In [1]:
import tensorflow as tf
import keras

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
from keras.layers import Dense,LSTM
import numpy as np 
import json

In [4]:
with open ('dataset_seq.json','r') as file:
    dataset=json.load(file)

In [5]:
dataset

[{'question': 'Eh, udah makan belum?',
  'answer': 'Belum, lagi bingung mau makan apa. Ada ide?'},
 {'question': 'Gimana film yang kemarin kamu tonton?',
  'answer': 'Seru sih, ceritanya nggak ketebak. Tapi ending-nya agak gantung.'},
 {'question': 'Kamu suka kopi apa teh?',
  'answer': 'Kopi dong, tapi harus yang nggak terlalu pahit.'},
 {'question': 'Lagi dengerin lagu apa sekarang?',
  'answer': 'Lagi suka banget sama lagunya Coldplay yang baru. Kamu?'},
 {'question': 'Udah ngerjain tugas kelompok belum?',
  'answer': 'Belum, kayaknya bakal aku selesain besok deh. Deadline-nya kapan sih?'},
 {'question': 'Main game apa sekarang?',
  'answer': 'Lagi asik main Genshin Impact. Kamu main juga nggak?'},
 {'question': 'Weekend ini mau kemana?',
  'answer': 'Belum ada rencana. Kalau kamu ngajak, aku ikut!'},
 {'question': 'Kamu suka kucing atau anjing?',
  'answer': 'Dua-duanya sih, tapi kucing lebih lucu buat dipelihara.'},
 {'question': 'Pernah coba masak sendiri?',
  'answer': 'Iya, tap

In [6]:
Users = []
response = []
X_datasets = []
for i in range(len(dataset)) :
    Users.append(dataset[i]['question'])
    response.append(dataset[i]['answer'])
    X_datasets.append(dataset[i]['question'])
    X_datasets.append(dataset[i]['answer'])

In [7]:
tokenize = Tokenizer()
tokenize.fit_on_texts(X_datasets)

In [8]:
X_trains = tokenize.texts_to_sequences(Users)
Y_Trains = tokenize.texts_to_sequences(response)

In [9]:
Y_Trains

[[71, 22, 173, 16, 70, 4, 13, 174],
 [57, 24, 284, 3, 285, 7, 286, 89, 287, 288],
 [129, 176, 7, 74, 5, 3, 48, 290],
 [22, 15, 9, 25, 291, 292, 5, 107, 2],
 [71, 130, 26, 1, 296, 297, 131, 298, 89, 178, 24],
 [22, 299, 58, 300, 301, 2, 58, 41, 3],
 [71, 13, 304, 6, 2, 179, 1, 180],
 [306, 307, 24, 7, 181, 27, 108, 75, 308],
 [90, 7, 309, 310, 133, 60],
 [183, 311, 7, 1, 135, 312, 109, 38, 313, 24, 12, 314],
 [59, 175, 317, 318, 108, 9, 75, 185],
 [3, 109, 24, 10, 186, 187, 319, 2, 72],
 [61, 34, 61, 321, 57, 35, 75, 322],
 [16, 9, 69, 137, 3, 36, 36, 62, 138],
 [90, 326, 9, 3, 327, 110, 92],
 [15, 7, 190, 328, 5, 329, 35, 14],
 [8, 176, 134, 76, 44, 330, 331, 332, 333, 57, 9],
 [141, 36, 36, 192, 336, 19, 111, 193, 12, 194, 60],
 [10, 42, 42, 195, 339, 22, 340, 112],
 [341, 25, 342, 196, 3, 343, 5, 1, 344],
 [347, 20, 348, 197, 349, 9, 350, 351, 89, 41, 91],
 [58, 132, 19, 177, 140, 94, 353, 198, 95],
 [8, 59, 63, 200, 7, 10, 357, 358, 359, 35],
 [61, 90, 6, 362, 113, 363, 364, 7, 3, 1

In [11]:
temp_length_data = tokenize.texts_to_sequences(X_datasets)
max_length_data = max([len(x) for x in temp_length_data])

In [16]:
X_trains = pad_sequences(X_trains,padding='post',maxlen=max_length_data)
Y_Trains = pad_sequences(Y_Trains,padding='post',maxlen=max_length_data)

In [17]:
X_decoded_trains = pad_sequences(Y_Trains,padding='post',maxlen=max_length_data)

In [21]:
for i in range(len(X_decoded_trains)) :
    X_decoded_trains[i][0] = 0

In [31]:
dimention = len(tokenize.index_word) + 1

In [32]:
dimention

715

In [83]:
encoder_input = keras.Input(shape=(max_length_data,))
encoder_Embbeding = keras.layers.Embedding(input_dim=dimention,output_dim=128)(encoder_input)
encoder_lstm = LSTM(64,return_state=True)
encoder_outputs,state_h,state_c= encoder_lstm(encoder_Embbeding)
states =[state_h,state_c]
decoder_input = keras.Input(shape=(None,))
decoder_Embedding = keras.layers.Embedding(input_dim=dimention,output_dim=128)(decoder_input)
decoder_lstm = LSTM(64,return_state=True,return_sequences=True)
decoder_outputs,_,_ = decoder_lstm(decoder_Embedding,initial_state=states)
decoder_dense = Dense(dimention, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [84]:
Encoder_decoder = keras.Model([encoder_input,decoder_input],decoder_outputs)

In [85]:
Encoder_decoder.compile(loss='sparse_categorical_crossentropy',optimizer=keras.optimizers.Adam(learning_rate=0.01),metrics=['accuracy'])

In [86]:
Encoder_decoder.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_24      │ (None, 22)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_25      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_15        │ (None, 22, 128)   │     91,520 │ input_layer_24[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_16        │ (None, None, 128) │     91,520 │ input_layer_25[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_19 (LSTM)      │ [(None, 64),      │     49,408 │ embedding_15[0][… │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_20 (LSTM)      │ [(None, None,     │     49,408 │ embedding_16[0][… │
│                     │ 64), (None, 64),  │            │ lstm_19[0][1],    │
│                     │ (None, 64)]       │            │ lstm_19[0][2]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, None, 715) │     46,475 │ lstm_20[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 328,331 (1.25 MB)

 Trainable params: 328,331 (1.25 MB)

 Non-trainable params: 0 (0.00 B)

In [88]:
Encoder_decoder.fit([X_trains,X_decoded_trains],Y_Trains,epochs=7,verbose=1)

Epoch 1/7
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.8275 - loss: 0.9972
Epoch 2/7
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.8503 - loss: 0.9079
Epoch 3/7
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.8559 - loss: 0.8832
Epoch 4/7
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.8815 - loss: 0.8248
Epoch 5/7
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.8936 - loss: 0.7660
Epoch 6/7
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.9073 - loss: 0.7206
Epoch 7/7
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.9232 - loss: 0.6577


In [89]:
Encoder_decoder.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_24      │ (None, 22)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_25      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_15        │ (None, 22, 128)   │     91,520 │ input_layer_24[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_16        │ (None, None, 128) │     91,520 │ input_layer_25[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_19 (LSTM)      │ [(None, 64),      │     49,408 │ embedding_15[0][… │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_20 (LSTM)      │ [(None, None,     │     49,408 │ embedding_16[0][… │
│                     │ 64), (None, 64),  │            │ lstm_19[0][1],    │
│                     │ (None, 64)]       │            │ lstm_19[0][2]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, None, 715) │     46,475 │ lstm_20[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 984,995 (3.76 MB)

 Trainable params: 328,331 (1.25 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 656,664 (2.50 MB)

In [90]:
Encoder_models = keras.Model(inputs=encoder_input, 
                             outputs=states)

In [91]:
decoder_state_h_input = keras.Input(shape=(64,))
decoder_state_c_input = keras.Input(shape=(64,))
decoder_all_states_input = [decoder_state_h_input,decoder_state_c_input]

In [93]:
decoder_outputs,state_h,state_c = decoder_lstm(decoder_Embedding,initial_state=decoder_all_states_input)
decoder_states = [state_h,state_c]
decoder_outputs= decoder_dense(decoder_outputs)

In [94]:
decoder_models = keras.Model([decoder_input] + decoder_all_states_input,
                             [decoder_outputs] + decoder_states)

In [95]:
decoder_models.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_25      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_16        │ (None, None, 128) │     91,520 │ input_layer_25[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_26      │ (None, 64)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_27      │ (None, 64)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_20 (LSTM)      │ [(None, None,     │     49,408 │ embedding_16[0][… │
│                     │ 64), (None, 64),  │            │ input_layer_26[0… │
│                     │ (None, 64)]       │            │ input_layer_27[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, None, 715) │     46,475 │ lstm_20[1][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 187,403 (732.04 KB)

 Trainable params: 187,403 (732.04 KB)

 Non-trainable params: 0 (0.00 B)

In [170]:
def sequences(seq_models):
    # Tokenisasi dan padding input
    pre_predict = tokenize.texts_to_sequences(seq_models)
    pre_predict = pad_sequences(pre_predict, maxlen=max_length_data, padding='post')

    # Prediksi states dari encoder
    states_predicted = Encoder_models.predict(pre_predict)
    print("Dimensi states_predicted:", len(states_predicted), len(states_predicted[0]))

    run_condition = True
    output_tokens = ""
    token_stop = list()

    # Inisialisasi target sequence dengan start token (dimensi batch_size=1, seq_len=1)
    target_seq = np.zeros((23,23))
    target_seq[0, 0] = 0  # Start token, misalnya 1

    print("Target sequence awal:", target_seq)
    while run_condition:
        outputs, h, c = decoder_models.predict([target_seq] + states_predicted)
        sample_token = np.argmax(outputs[0, -1, :])
        token_stop.append(sample_token)
        for index, value in tokenize.word_index.items():
            if index == sample_token:
                output_tokens += " " + value

        if len(token_stop) == max_length_data:
            run_condition = False

        # Update target sequence dan states untuk langkah berikutnya
        target_seq = np.zeros((23,23))
        target_seq[0, sample_token] = 0
        states_predicted = [h, c]
        print(sample_token)

    return output_tokens




In [98]:
X_trains[0]

array([18, 69, 70, 71,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0])

In [171]:
data1 = sequences("Kamu suka Kopi apa teh?")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
Dimensi states_predicted: 2 23
Target sequence awal: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

In [177]:
def sequencess(seq_models):
    # Tokenisasi dan padding input
    pre_predict = tokenize.texts_to_sequences(seq_models)
    pre_predict = pad_sequences(pre_predict, maxlen=max_length_data, padding='post')

    # Prediksi states dari encoder
    states_predicted = Encoder_models.predict(pre_predict)
    print("Dimensi states_predicted:", len(states_predicted), len(states_predicted[0]))

    run_condition = True
    output_tokens = ""
    token_stop = list()

    # Inisialisasi target sequence dengan start token (dimensi batch_size=1, seq_len=1)
    target_seq = np.zeros((24, 1))  # Batch size=1 dan seq_len=1 pada langkah pertama
    target_seq[0, 0] = 1  # Misalnya 1 adalah start token

    print("Target sequence awal:", target_seq)

    while run_condition:
        # Prediksi output dari decoder
        # Pastikan input ke decoder sesuai: [target_seq] + states_predicted
        outputs, h, c = decoder_models.predict([target_seq] + states_predicted)
        
        # Ambil token yang diprediksi
        sample_token = np.argmax(outputs[0, -1, :])

        # Tambahkan token yang diprediksi ke output
        token_stop.append(sample_token)

        # Mengubah token menjadi kata berdasarkan word_index
        for index, value in tokenize.word_index.items():
            if index == sample_token:
                output_tokens += " " + value

        # Jika panjang token yang diprediksi mencapai batas, keluar dari loop
        if len(token_stop) == max_length_data:
            run_condition = False

        # Perbarui target_seq untuk langkah berikutnya
        target_seq = np.zeros((24, 1))  # Update ke dimensi (1, 1) setiap langkah
        target_seq[0, 0] = sample_token  # Token berikutnya yang dipilih

        # Perbarui states_predicted dengan hidden dan cell states untuk langkah berikutnya
        states_predicted = [h, c]
        print("Sample token:", sample_token)

    return output_tokens



In [178]:
sequencess("kamu suka kopi apa teh ?")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Dimensi states_predicted: 2 24
Target sequence awal: [[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 501ms/step
Sample token: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
Sample token: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
Sample token: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Sample token: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
Sample token: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Sample token: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Sample token: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Sample token: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Sample token: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Sample token: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Sample token: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
Sample token: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Sample token: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Sample token: 1
1/1 ━━━

''